<a href="https://colab.research.google.com/github/shanshanli806/Artificial-Intelligence/blob/main/Assignment02/CSc448_A02.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing libraries
import numpy as np
import pandas as pd

## Part 1. Loading the dataset

In [2]:
# Using pandas load the dataset (Load remotely, not locally)
df = pd.read_csv('https://raw.githubusercontent.com/profmcnich/example_notebook/main/science_data_large.csv')

# Output the first 15 rows of the data
df.head(15)

,Temperature °C,Mols KCL,Size nm^3
0,469,647,6.244743e+05
1,403,694,5.779610e+05
2,302,975,6.196847e+05
3,779,916,1.460449e+06
4,901,18,4.325726e+04
5,545,637,7.124634e+05
6,660,519,7.006960e+05
7,143,869,2.718260e+05
8,89,461,8.919803e+04
9,294,776,4.770210e+05


In [3]:
# Display a summary of the table information (number of datapoints, etc.)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature °C  1000 non-null   int64  
 1   Mols KCL        1000 non-null   int64  
 2   Size nm^3       1000 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 23.6 KB


In [4]:
df.shape # (rows, colums)

(1000, 3)

In [5]:
df.describe()

,Temperature °C,Mols KCL,Size nm^3
count,1000.000000,1000.000000,1.000000e+03
mean,500.500000,471.530000,5.086111e+05
std,288.819436,288.482872,4.474838e+05
min,1.000000,1.000000,1.611429e+01
25%,250.750000,226.750000,1.298267e+05
50%,500.500000,459.500000,3.827182e+05
75%,750.250000,710.250000,7.603211e+05
max,1000.000000,1000.000000,1.972127e+06


## Part 2. Splitting the dataset
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [6]:
# Printing the columns in the dataset
df.columns

Index(['Temperature °C', 'Mols KCL', 'Size nm^3'], dtype='object')

In [7]:
# Take the pandas dataset and split it into our features (X) and label (y)
# features (X)
X = df[['Temperature °C', 'Mols KCL']]
# label (y)
y = df['Size nm^3']

In [8]:
# Use skLearn to split the features and labels into a training/test set. (90% train, 10% test)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1)

In [9]:
X_train.shape, y_train.shape

((900, 2), (900,))

In [10]:
X_test.shape, y_test.shape

((100, 2), (100,))

## Part 3. Perform a Linear Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression

In [11]:
# Use sklearn to train a model on the training set
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [12]:
# Create a sample datapoint and predict the output of that sample with the trained model
y_pred = reg.predict(X_test)
pd.DataFrame({"Actual" : y_test, 'Predicted' : y_pred})

,Actual,Predicted
235,1.231728e+05,1.557036e+05
299,4.086765e+05,6.432282e+05
890,1.335314e+05,1.019581e+05
984,1.279600e+06,1.104241e+06
737,8.687293e+05,8.318090e+05
...,...,...
375,1.884383e+05,3.045494e+05
925,9.762603e+04,2.154767e+05
587,1.156133e+05,5.155103e+05
170,2.711203e+04,-1.231032e+05


In [13]:
# Report on the score for that model, in your own words (markdown, not code) explain what the score means
score = r2_score(y_test, y_pred)
print("The score for the model is", score)

The score for the model is 0.8417350599872937


`model.score(X_test, y_test)` is the prediction for `X_test` and compares it with `y_test`, and returns the R-squared value for the model. 

R-squared is a measure of how close the data are to the fitted regression line. More specifically, it is a statistical measure of the percentage of the variance in the dependent variable that is predicted from the independent variables.

In [14]:
print("Here, the accuracy for this model is {0:.2%}".format(score))

Here, the accuracy for this model is 84.17%


In [15]:
# coefficents
coef = reg.coef_
pd.DataFrame(coef, X.columns, columns=['Coefficient'])

,Coefficient
Temperature °C,880.420978
Mols KCL,1029.433011


In [16]:
intercept = reg.intercept_
print("The intercept is", intercept)

The intercept is -414919.5383346601


In [17]:
# Extract the coefficents and intercept from the model and write an equation for your h(x) using LaTeX
from IPython.display import Latex

Latex(f"""
\\begin{{equation*}}
\\ h(x) = {{{intercept}}} + {{{coef[0]}}}a + {{{coef[1]}}}b
\\end{{equation*}}
""")

<IPython.core.display.Latex object>

## Part 4. Use Cross Validation

https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation

In [18]:
# Use the cross_val_score function to repeat your experiment across many shuffles of the data
# Report on their finding and their significance

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=0)
cv_results = cross_val_score(reg, X_test, y_test, cv=cv)
print("Cross Val scores :", cv_results)

Cross Val scores : [0.79570805 0.91562328 0.82089814 0.88481862 0.83904214]


In [19]:
print("%0.4f accuracy with a standard deviation of %0.4f" % (cv_results.mean(), cv_results.std()))

0.8512 accuracy with a standard deviation of 0.0434


Cross validation creates multiple random splits of the dataset into training and validation data. Here, we splitted the data into 5 groups. The dataset are divided into 5 equal parts. During each split, a group of data is holdout as the test set and take the remaining groups as a training set. Then, a model is fit to the training set and evaluate it on the test set, and predictive accuracy is assessed using the validation data.

## Part 5. Using Polynomial Regression

https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions

In [20]:
# Using the PolynomialFeatures library perform another regression on an augmented dataset of degree 2
# Report on the metrics and output the resultant equation as you did in Part 3.

from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

# applying polynomial regression degree 2
poly = PolynomialFeatures(degree=2)
X_train_trans = poly.fit_transform(X_train)
X_test_trans = poly.transform(X_test)

In [21]:
X_train_trans.shape, X_test_trans.shape

((900, 6), (100, 6))

In [22]:
# include bias parameter
reg_new = LinearRegression()
reg_new.fit(X_train_trans, y_train)

LinearRegression()

In [23]:
# prediction
y_pred_new = reg_new.predict(X_test_trans)
pd.DataFrame({"Actual" : y_test, 'Predicted' : y_pred_new})

,Actual,Predicted
235,1.231728e+05,1.231728e+05
299,4.086765e+05,4.086765e+05
890,1.335314e+05,1.335314e+05
984,1.279600e+06,1.279600e+06
737,8.687293e+05,8.687293e+05
...,...,...
375,1.884383e+05,1.884383e+05
925,9.762603e+04,9.762603e+04
587,1.156133e+05,1.156133e+05
170,2.711203e+04,2.711203e+04


In [24]:
# score
score = r2_score(y_test, y_pred_new)
print("The score for the model is", score)

The score for the model is 1.0


In [25]:
print("Here, the accuracy for this model is {0:.2%}".format(score))

Here, the accuracy for this model is 100.00%


In [26]:
# coefficents
coef_new = reg_new.coef_
print("The coefficients are ", coef_new)

The coefficients are  [ 0.00000000e+00  1.20000000e+01 -1.26501034e-07  1.15356613e-11
  2.00000000e+00  2.85714287e-02]


In [27]:
coef_new.shape

(6,)

In [28]:
# intercept
intercept_new = reg_new.intercept_
print("The intercept is", intercept_new)

The intercept is 1.925509423017502e-05


In [29]:
# rounding all numbers to 2 decimal places
Latex(f"""
\\begin{{equation*}}
\\ h(x) = {{{intercept_new}}} + {{{coef_new[0]}}} + {{{coef_new[1]}}}a + {{{coef_new[2]}}}b + {{{coef_new[3]}}}a^2 + {{{coef_new[4]}}}ab + {{{coef_new[5]}}}b^2
\\end{{equation*}}
""")

<IPython.core.display.Latex object>

Here, we convert the input to polynomial   terms in degree 2, and fit the linear regression as polynomial regression. We can see that the polynomial regression have a R-squared score of 1.0, which indicates that the accuracy for this model is 100%, as compared to the linear line. Unlike, linear regression, polynomial regression have a board range of function that can fit under it and fits a wide range of curvatures. Therefore, the polynomial regression fit the data easily into a model than for a linear regression in part 3.